Installation
```
pip install spacy_experimental
pip install chardet
pip install thinc[torch]
pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.1/en_coreference_web_trf-3.4.0a2-py3-none-any.whl
```


In [1]:

import spacy
nlp = spacy.load("en_coreference_web_trf")

In [2]:
doc = nlp("My dad passed away last summer after suffering from the behavioral variant FTD, he was older when he was diagnosed but he probably had it longer but we probably missed a lot of signs over the years. When the symptoms really started getting bad that's when my dad's doctor ordered an MRI and we learned that the frontal part of his brain was shrinking and in atrophy.  For my dad, from diagnosis to death it took less than two years, a year and nine and a half months to be exact.    As another poster mentioned I wrote a journal of everything he went through and it was challenging for sure as my mother and I were his caregivers the entire time.  If there was one blessing it was that he never lost his memory of who my mom and I were so that was a good thing.  I could write a book here on what we went through with this disease but almost seven months since his passing, I wish I could take care of him for just one more day.")
print(doc.spans)

{'coref_clusters_1': [My dad, he, he, he, my dad's, his, my dad, he, his, he, his, his, him], 'coref_clusters_2': [the behavioral variant FTD, it, this disease], 'coref_clusters_3': [passed, that, diagnosis to death, his passing], 'coref_clusters_4': [My, my, my, I, my, I, my, I, I, I, I], 'coref_clusters_5': [wrote, it], 'coref_clusters_6': [one blessing, it], 'coref_clusters_7': [my mother, my mom, we], 'coref_clusters_8': [lost, that]}


In [3]:
doc = nlp("Dr. Allison, mom is behaving the opposite way. She lives in an assisted living facility and she is pushing the call button every few minutes to have them hand her the remote when it is sitting right next to her, wanting them to wipe her when she goes to the bathroom and many other things like that. The caregivers are so frustrated and the nurse is trying to get her to do these things for herself while she still can. I am frustrated and can’t be around her because I am just so exhausted and I feel like her slave. What is your advice?")
print(doc.spans)

{'coref_clusters_1': [Dr. Allison, mom, She, she, her, her, her, she, her, herself, she, I, her, I, I, her, your], 'coref_clusters_2': [the remote, it], 'coref_clusters_3': [them, them]}


In [5]:
doc = nlp("""Yes but it all depends on the state of dementia of your LO. Anything I try to say, request her to do or discuss with my mother is met with hostility. She is deeply paranoid and suspicious. She makes unfounded and quite horrid accusations to and about me. She argues with me even when I am being nice to her. She argues about her arguing!!! Bascially I am her punch bag and it is soul destroying. If you haven't already get a PoA and do what you have to do without them making it harder for you. If day to day tasks become impossible to complete then get home help for assistance or check your LO into a care facility. There comes a point when the stress and aggravation is just not worth it. They have dementia what do they know? Many times I feel like my mother is frying my brain and it literally hurts my head and I just want out. Can't do it any more, but I am trapped and she knows it because as well as having dementia she is a manipulative selfish narcissist which is a terrible combination. Get help is all I can say. Trying to deal with contentious issues on a one to one basis with a LO who is resistant non compliant is, 99% of the time, going to fail.""")
print(doc.spans)

{'coref_clusters_1': [I, my, me, me, I, I, I, my, my, my, I, I, I], 'coref_clusters_2': [her, my mother, She, She, She, her, She, her, her, my mother, she, she], 'coref_clusters_3': [am, it], 'coref_clusters_4': [They, they], 'coref_clusters_5': [frying, it], 'coref_clusters_6': [am, it]}


In [8]:
doc = nlp("""We embraced my husband's dementia because.... it is what it is.   When he first got the diagnosis we were mostly relieved, better than the dramas and psychosis that plagued him for a few years.   That had been a miserable time and I was his target.   Knowing what it was,  made our lives better.

We ended up with a good medical team, the right meds after a few months of trials, government pension,  did all the legal paperwork while he could still function and we told people what he had with no shame or hesitation.

7 years down with him at a moderate to severe stage and him sitting most days with his own thoughts, I think maybe his life is not so bad, no decisions, no bills, no driving, no responsibility,  not answerable for anything...... perpetual holiday of the mind.  It's then I think he's the lucky one.   I just get the work.
"""
)
print(doc.spans)

{'coref_clusters_1': [my husband's dementia, it, it, it], 'coref_clusters_2': [my husband's, he, him, his, he, he, him, him, his, his, he], 'coref_clusters_3': [We, we, our, We, we], 'coref_clusters_4': [my, I, I, I, I]}


### Proposed Methodology
1. Sort most common head coref clusters of interest (start w/ my then a word...). Count them, calculate distribution of length of references.
2. Create list of relevant coref cluster heads.
3. Determine threshold for "talking about self". What is length of "I" (knowledgeable informant) vs. the patient?

### Proposed NLP Pipeline
1. Remove single sentence comments
2. NER, replace names
3. Remove thank you's
4. Apply coreference pipeline

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
assert "transformer" not in nlp.pipe_names
nlp_coref = spacy.load("en_coreference_web_trf")
nlp.add_pipe("transformer", source=nlp_coref)
nlp.add_pipe("coref", source=nlp_coref)


c:\Users\david\anaconda3\envs\spacy_experimental\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
doc = nlp("""We embraced John Roberts' dementia because.... it is what it is.   When he first got the diagnosis we were mostly relieved, better than the dramas and psychosis that plagued him for a few years.   That had been a miserable time and I was his target.   Knowing what it was,  made our lives better.

We ended up with a good medical team, the right meds after a few months of trials, government pension,  did all the legal paperwork while he could still function and we told people what he had with no shame or hesitation.

7 years down with him at a moderate to severe stage and him sitting most days with his own thoughts, I think maybe his life is not so bad, no decisions, no bills, no driving, no responsibility,  not answerable for anything...... perpetual holiday of the mind.  It's then I think he's the lucky one.   I just get the work.

Dr. PERSON, thank you so much!
"""
)
print(doc.spans)

{'coref_head_clusters_1': [dementia, it, it, it], 'coref_head_clusters_2': [Roberts, he, him, his, he, he, him, him, his, his, he], 'coref_head_clusters_3': [We, we, our, We, we], 'coref_head_clusters_4': [I, I, I, I, PERSON, you]}


In [4]:
# named entity replacement
#for s in doc.sents:print(s)
for e in doc.ents:
    print(e.text, e.label_)

John Roberts' PERSON
first ORDINAL
a few years DATE
a few months DATE
7 years DATE
